In [1]:
from datetime import datetime
from os import environ

from boto3 import client
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from model_registry import ModelRegistry
from model_registry.utils import s3_uri_from
from numpy import load, save
import onnx
from pandas import read_csv
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler
from tf2onnx import convert

2024-11-11 17:18:06.608016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-11 17:18:06.608068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-11 17:18:06.609215: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 17:18:06.615322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-11 17:18:07.404620: W tensorflow/compiler/tf2

## Reading data connection

In [2]:
s3_endpoint_url = environ.get('AWS_S3_ENDPOINT')
s3_access_key = environ.get('AWS_ACCESS_KEY_ID')
s3_secret_key = environ.get('AWS_SECRET_ACCESS_KEY')
s3_bucket_name = environ.get('AWS_S3_BUCKET')

# Data ingestion

In [3]:
print(f'Downloading data "training-data.csv" '
      f'from bucket "{s3_bucket_name}" '
      f'from S3 storage at {s3_endpoint_url}')

s3_client = client(
    's3', endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key, aws_secret_access_key=s3_secret_key
)

s3_client.download_file(
    s3_bucket_name,
    'data/training-data.csv',
    './data/raw_data.csv'
)

# Data preprocessing

In [4]:
df = read_csv('./data/raw_data.csv')

rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(
    df['Amount'].values.reshape(-1, 1)
)
df['scaled_time'] = rob_scaler.fit_transform(
    df['Time'].values.reshape(-1, 1)
)
df.drop(['Time', 'Amount'], axis=1, inplace=True)
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

X = df.drop('Class', axis=1)
y = df['Class']
sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain = X.iloc[train_index]
    original_ytrain = y.iloc[train_index]

original_Xtrain = original_Xtrain.values
original_ytrain = original_ytrain.values

sm = SMOTE(sampling_strategy='minority', random_state=42)
Xsm_train, ysm_train = sm.fit_resample(original_Xtrain, original_ytrain)

save('./data/training_samples.npy', Xsm_train)
save('./data/training_labels.npy', ysm_train)

Train: [ 30473  30496  31002 ... 284804 284805 284806] Test: [    0     1     2 ... 57017 57018 57019]
Train: [     0      1      2 ... 284804 284805 284806] Test: [ 30473  30496  31002 ... 113964 113965 113966]
Train: [     0      1      2 ... 284804 284805 284806] Test: [ 81609  82400  83053 ... 170946 170947 170948]
Train: [     0      1      2 ... 284804 284805 284806] Test: [150654 150660 150661 ... 227866 227867 227868]
Train: [     0      1      2 ... 227866 227867 227868] Test: [212516 212644 213092 ... 284804 284805 284806]


# Model training

In [5]:
epoch_count = 20
learning_rate = 0.001

Xsm_train = load('./data/training_samples.npy')
ysm_train = load('./data/training_labels.npy')
n_inputs = Xsm_train.shape[1]

oversample_model = Sequential([
    Dense(n_inputs, input_shape=(n_inputs, ), activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax'),
])
oversample_model.compile(
    Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)
training_metrics = oversample_model.fit(
    Xsm_train,
    ysm_train,
    validation_split=0.2,
    batch_size=300,
    epochs=epoch_count,
    shuffle=True,
    verbose=2,
)
accuracy = training_metrics.history['accuracy'][-1]
print(f'finished training model with final accuracy score of {accuracy}')
onnx_model, _ = convert.from_keras(oversample_model)
onnx.save(onnx_model, 'model.onnx')

Epoch 1/20
1214/1214 - 2s - loss: 0.0720 - accuracy: 0.9724 - val_loss: 0.0350 - val_accuracy: 0.9890 - 2s/epoch - 1ms/step
Epoch 2/20
1214/1214 - 1s - loss: 0.0174 - accuracy: 0.9953 - val_loss: 0.0171 - val_accuracy: 0.9985 - 1s/epoch - 930us/step
Epoch 3/20
1214/1214 - 1s - loss: 0.0102 - accuracy: 0.9978 - val_loss: 0.0112 - val_accuracy: 0.9993 - 1s/epoch - 929us/step
Epoch 4/20
1214/1214 - 1s - loss: 0.0073 - accuracy: 0.9984 - val_loss: 0.0182 - val_accuracy: 0.9960 - 1s/epoch - 916us/step
Epoch 5/20
1214/1214 - 1s - loss: 0.0055 - accuracy: 0.9988 - val_loss: 0.0037 - val_accuracy: 1.0000 - 1s/epoch - 929us/step
Epoch 6/20
1214/1214 - 1s - loss: 0.0044 - accuracy: 0.9991 - val_loss: 0.0020 - val_accuracy: 0.9999 - 1s/epoch - 934us/step
Epoch 7/20
1214/1214 - 1s - loss: 0.0037 - accuracy: 0.9993 - val_loss: 0.0014 - val_accuracy: 1.0000 - 1s/epoch - 933us/step
Epoch 8/20
1214/1214 - 1s - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.0017 - val_accuracy: 1.0000 - 1s/epoch - 933us

2024-11-11 17:18:34.589530: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-11-11 17:18:34.589642: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-11-11 17:18:34.613114: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-11-11 17:18:34.613261: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


# Model upload

In [7]:
model_registry_endpoint_url = 'https://model-registry-rest.apps.cluster-4g5bb.4g5bb.sandbox1112.opentlc.com'
timestamp = datetime.now().strftime('%y%m%d%H%M')
model_object_name = f'models/model-{timestamp}.onnx'

try:
    s3_client.upload_file('model.onnx', s3_bucket_name, model_object_name)
except Exception:
    print(f'S3 upload to bucket {s3_bucket_name} at {s3_endpoint_url} failed!')
    raise
print(f'model uploaded and available as "{model_object_name}"')

registry = ModelRegistry(
    server_address=model_registry_endpoint_url,
    port=443,
    author='user'
)

model_description = '''
Shallow neural network trained on Credit Card Fraud Detector dataset 
(https://www.kaggle.com/code/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets).\n
Deployed model expects input vector of shape [1, 30] with FP32-type values, 
returns vector of shape [1, 2] with FP32-type values denoting predicted 
probabilities for non-fraud / fraud. See sample:
https://github.com/mamurak/os-mlops/blob/main/notebooks/fraud-detection-onnx/online-scoring.ipynb
'''
registry.register_model(
    'fraud-detection',
    uri=s3_uri_from(model_object_name, s3_bucket_name),
    version=timestamp,
    description=model_description,
    model_format_name='onnx',
    model_format_version='1',
    storage_key='aws-connection-fraud-detection',
    metadata={
        'epoch_count': str(epoch_count),
        'learning_rate': str(learning_rate),
        'accuracy': str(accuracy),
        'fraud-detection': '',
        'onnx': '',
    }
)

model uploaded and available as "models/model-2411111719.onnx"


RegisteredModel(id='1', description=None, external_id=None, create_time_since_epoch='1731345575357', last_update_time_since_epoch='1731345575357', custom_properties=None, name='fraud-detection', owner='user', state=<RegisteredModelState.LIVE: 'LIVE'>)